In [ ]:
from ultralytics import YOLO

# 加载模型
yolo_model = YOLO('yolov8m.pt')  # 使用合适的模型权重

# 获取模型类别名称
class_names = yolo_model.names  # 这是一个字典，通常形式是{index: 'class_name'}

# 打印类别名称与索引
for index, name in class_names.items():
    print(f"Index: {index}, Name: {name}")

Index: 0, Name: person
Index: 1, Name: bicycle
Index: 2, Name: car
Index: 3, Name: motorcycle
Index: 4, Name: airplane
Index: 5, Name: bus
Index: 6, Name: train
Index: 7, Name: truck
Index: 8, Name: boat
Index: 9, Name: traffic light
Index: 10, Name: fire hydrant
Index: 11, Name: stop sign
Index: 12, Name: parking meter
Index: 13, Name: bench
Index: 14, Name: bird
Index: 15, Name: cat
Index: 16, Name: dog
Index: 17, Name: horse
Index: 18, Name: sheep
Index: 19, Name: cow
Index: 20, Name: elephant
Index: 21, Name: bear
Index: 22, Name: zebra
Index: 23, Name: giraffe
Index: 24, Name: backpack
Index: 25, Name: umbrella
Index: 26, Name: handbag
Index: 27, Name: tie
Index: 28, Name: suitcase
Index: 29, Name: frisbee
Index: 30, Name: skis
Index: 31, Name: snowboard
Index: 32, Name: sports ball
Index: 33, Name: kite
Index: 34, Name: baseball bat
Index: 35, Name: baseball glove
Index: 36, Name: skateboard
Index: 37, Name: surfboard
Index: 38, Name: tennis racket
Index: 39, Name: bottle
Index: 

In [ ]:
import os
import shutil
from ultralytics import YOLO
from PIL import Image

def create_folder(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def get_all_image_paths(folder_path):
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
    all_image_paths = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                all_image_paths.append(os.path.join(root, file))
    return all_image_paths

def iou(box1, box2):
    """计算两个边界框之间的交并比（IoU）"""
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2
    inter_x1 = max(x1, x3)
    inter_y1 = max(y1, y3)
    inter_x2 = min(x2, x4)
    inter_y2 = min(y2, y4)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x4 - x3) * (y4 - y3)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

def save_boxes_to_file(boxes, file_path):
    """将边界框参数保存到文件"""
    with open(file_path, 'w') as f:
        for box in boxes:
            f.write(','.join(map(str, box)) + '\n')

def detect_and_classify_images(base_path, output_base_path):
    yolo_model_person = YOLO('yolov8m.pt')  # 使用预训练的YOLOv8模型
    clothing_model = YOLO('deepfashion2_yolov8s-seg.pt')  # 衣服检测模型

    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        if not os.path.isdir(folder_path):
            continue

        flat_clothing_dir = os.path.join(output_base_path, folder_name, '平铺服装')
        human_clothing_dir = os.path.join(output_base_path, folder_name, '人身服装')
        not_qualify_dir = os.path.join(output_base_path, folder_name, '不满足要求')

        create_folder(flat_clothing_dir)
        create_folder(human_clothing_dir)
        create_folder(not_qualify_dir)

        image_paths = get_all_image_paths(folder_path)

        for image_path in image_paths:
            image = Image.open(image_path)
            image_area = image.size[0] * image.size[1]

            print(f"\nProcessing image: {os.path.basename(image_path)}")

            # Step 1: 检测人
            results_person = yolo_model_person(image_path, classes=[0], device='cuda')
            person_boxes = [box.xyxy.tolist()[0] for result in results_person for box in result.boxes]

            if person_boxes:
                largest_person_box = max(person_boxes, key=lambda box: (box[2] - box[0]) * (box[3] - box[1]))
                person_area = (largest_person_box[2] - largest_person_box[0]) * (largest_person_box[3] - largest_person_box[1])

                person_area_ratio = person_area / image_area
                print(f"Detected person boxes: {person_boxes}")
                print(f"Largest person box area ratio: {person_area_ratio:.2f}")

                iou_values = [iou(largest_person_box, other_box) for other_box in person_boxes if other_box != largest_person_box]
                print(f"Person IoUs with other boxes: {iou_values}")

                if person_area_ratio > 0.5 and all(value < 0.1 for value in iou_values):
                    image_name = os.path.basename(image_path)
                    shutil.copy(image_path, os.path.join(human_clothing_dir, image_name))
                    save_boxes_to_file([largest_person_box], os.path.join(human_clothing_dir, image_name.replace('.jpg', '_boxes.txt')))
                    print("Classified as human clothing")
                    continue

            # Step 2: 检测衣服
            results_clothing = clothing_model(image_path, device='cuda')
            clothing_boxes = [box.xyxy.tolist()[0] for result in results_clothing for box in result.boxes]

            if clothing_boxes:
                largest_clothing_box = max(clothing_boxes, key=lambda box: (box[2] - box[0]) * (box[3] - box[1]))
                clothing_area = (largest_clothing_box[2] - largest_clothing_box[0]) * (largest_clothing_box[3] - largest_clothing_box[1])

                clothing_area_ratio = clothing_area / image_area
                print(f"Detected clothing boxes: {clothing_boxes}")
                print(f"Largest clothing box area ratio: {clothing_area_ratio:.2f}")

                iou_values_clothing = [iou(largest_clothing_box, other_box) for other_box in clothing_boxes if other_box != largest_clothing_box]
                print(f"Clothing IoUs with other boxes: {iou_values_clothing}")

                if clothing_area_ratio > 0.5 and all(value < 0.1 for value in iou_values_clothing):
                    image_name = os.path.basename(image_path)
                    shutil.copy(image_path, os.path.join(flat_clothing_dir, image_name))
                    save_boxes_to_file([largest_clothing_box], os.path.join(flat_clothing_dir, image_name.replace('.jpg', '_boxes.txt')))
                    print("Classified as flat clothing")
                    continue

            # 如果都不符合
            image_name = os.path.basename(image_path)
            shutil.copy(image_path, os.path.join(not_qualify_dir, image_name))
            print("Classified as not qualified")

# 使用示例路径，请根据实际情况调整
base_path = '夹克'
output_base_path = '夹克1'
detect_and_classify_images(base_path, output_base_path)

100%|██████████| 49.7M/49.7M [31:14<00:00, 27.8kB/s]



Processing image: O1CN01JHOJMy1V3TMDPGAbH___2665222597.jpg

image 1/1 f:\\\\4878342940501\\O1CN01JHOJMy1V3TMDPGAbH___2665222597.jpg: 640x480 1 person, 43.3ms
Speed: 3.0ms preprocess, 43.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
Detected medium confidence clothing boxes: [([4.650688171386719, 19.7235107421875, 1490.79736328125, 1996.7803955078125], 0.39937758445739746)]
Largest clothing box area ratio: 0.98
Classified as flat clothing

Processing image: O1CN016RzuAF1V3TMC2zXdg___2665222597.jpg

image 1/1 f:\\\\4878342940501\\O1CN016RzuAF1V3TMC2zXdg___2665222597.jpg: 640x480 (no detections), 26.0ms
Speed: 3.0ms preprocess, 26.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
Classified as not qualified

Processing image: O1CN01h4oXtm1V3TMF9U3af___2665222597.jpg

image 1/1 f:\\\\4878342940501\\O1CN01h4oXtm1V3TMF9U3af___2665222597.jpg: 640x480 (no detections), 25.0ms
Speed: 3.0ms preprocess, 25.0ms inference, 2.0ms postprocess per image at sha

KeyboardInterrupt: 